In [197]:
%matplotlib inline
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import psycopg2
from dotenv import load_dotenv

In [198]:
print(f"Importando do arquivo .env: {load_dotenv(dotenv_path=r'../.env')}")

Importando do arquivo .env: True


In [199]:
#https://www.youtube.com/watch?v=MEEVcpnfElY
leituras_dique6c = pd.read_csv('../arquivos/leituras_dique6c.csv')
leituras_bvsa = pd.read_csv('../arquivos/leituras_bvsa.csv')

In [200]:
loc_bvsa = pd.read_csv('../arquivos/loc_bvsa.csv', sep=';', decimal=',')
loc_dique6c = pd.read_csv('../arquivos/loc_dique6c.csv', sep=';', decimal=',')

In [201]:
loc_sensores = pd.read_excel('../arquivos/LOCALIZAÇÃO_SENSORES.xlsx', engine='openpyxl', header=2)

In [202]:
leituras_dique6c['area'] = 'BVSA'
leituras_bvsa['area'] = 'Dique 6C'

In [203]:
leituras = pd.concat([leituras_bvsa, leituras_dique6c])
leituras = leituras.rename(columns={"Sensor":"sensor", "Data":"data", "Valor":"valor", "Unidade":"unidade", "Direcao_Saida":"direcao_saida", "Estado":"estado"})
leituras['direcao_saida'] = leituras['direcao_saida'].replace('Cota NA', 'Cota do NA')

In [204]:
leituras_bvsa['Sensor'] = leituras_bvsa['Sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').strip())
leituras_dique6c['Sensor'] = leituras_dique6c['Sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').strip())
loc_bvsa['Sensor'] = loc_bvsa['Sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').strip())
loc_dique6c['Sensor'] = loc_dique6c['Sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').strip())

In [205]:
leituras[leituras['direcao_saida'] == 'Cota do NA']['valor'].mean()

88.14659755901718

In [206]:
leituras['direcao_saida'].value_counts()

Cota Piezométrica                             19204
Carga Piezométrica                            19204
Vazão                                         10554
Cota do NA                                     6227
Recalque (Z)                                   1007
Deslocamento Longitudinal (X)                  1007
Deslocamento Transversal (Y)                   1007
Recalque                                        819
Deslocamento da Estaca Dir(-)/Esq(+)            819
Deslocamento do Afastamento Mont(-)/Jus(+)      819
Leitura                                          32
Name: direcao_saida, dtype: int64

In [207]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

leituras['id_unidade'] = le.fit_transform(leituras['unidade']) +1
leituras['id_direcao_saida'] = le.fit_transform(leituras['direcao_saida']) +1
leituras['id_estado'] = le.fit_transform(leituras['estado']) +1
leituras['id_area'] = le.fit_transform(leituras['area']) + 1

leituras = leituras.reset_index(drop=True)

In [208]:
leituras

,sensor,data,valor,unidade,direcao_saida,estado,area,id_unidade,id_direcao_saida,id_estado,id_area
0,PZ-BM-BV-54,2013-09-28 11:30:00,24.570,m,Cota Piezométrica,normal,Dique 6C,3,2,3,2
1,PZ-BM-BV-54,2013-09-28 11:30:00,0.000,m.c.a,Carga Piezométrica,normal,Dique 6C,4,1,3,2
2,PZ-BM-BV-53,2013-10-07 10:40:00,0.000,m.c.a,Carga Piezométrica,normal,Dique 6C,4,1,3,2
3,PZ-BM-BV-53,2013-10-07 10:40:00,27.740,m,Cota Piezométrica,normal,Dique 6C,3,2,3,2
4,PZ-BM-BV-53,2013-10-09 14:49:00,27.740,m,Cota Piezométrica,normal,Dique 6C,3,2,3,2
...,...,...,...,...,...,...,...,...,...,...,...
60694,MNA-SM-7541 (I),2022-04-13 14:04:00,87.950,m,Cota do NA,normal,BVSA,3,3,3,1
60695,MNA-SM-7542 (S),2022-04-13 14:09:00,74.720,m,Cota do NA,normal,BVSA,3,3,3,1
60696,MNA-SM-7542 (I),2022-04-13 14:09:00,71.000,m,Cota do NA,normal,BVSA,3,3,3,1
60697,MNA-SM-7539 (S),2022-04-13 14:46:00,100.400,m,Cota do NA,normal,BVSA,3,3,3,1


In [209]:
unidade = leituras[['id_unidade', 'unidade']].sort_values(by=['id_unidade']).drop_duplicates()
direcao_saida = leituras[['id_direcao_saida', 'direcao_saida']].sort_values(by=['id_direcao_saida']).drop_duplicates()
estado = leituras[['id_estado', 'estado']].sort_values(by=['id_estado']).drop_duplicates()
area = leituras[['id_area', 'area']].sort_values(by=['id_area']).drop_duplicates()

In [210]:
#Criação da tabela de sensores

In [211]:
aux = pd.concat([leituras_bvsa[['Sensor']], leituras_dique6c[['Sensor']], loc_bvsa[['Sensor']], loc_dique6c[['Sensor']]])
tabela_sensores = aux.drop_duplicates().sort_values(by=['Sensor']).reset_index(drop=True)
tabela_sensores['Sensor'] = tabela_sensores['Sensor'].apply(lambda x: x.replace("‐", '-').strip())
tabela_sensores.index = tabela_sensores.index+1

In [212]:
tabela_sensores.to_excel('../tabelas/sensores.xlsx')

In [213]:
dict_sensores = tabela_sensores.reset_index()[['Sensor','index']].to_dict(orient = 'tight')
dict_sensores = dict(dict_sensores['data']);

In [214]:
leituras['id_sensor'] = leituras['sensor'].map(dict_sensores) +1

In [215]:
leituras = leituras[['id_sensor','id_unidade', 'id_direcao_saida', 'id_estado', 'id_area', 'sensor', 'data', 'valor', 'unidade', 'direcao_saida', 'estado', 'area']]

In [216]:
leituras = leituras.drop_duplicates()

In [217]:
#Tratamento dos dados de localização
import geopandas as gpd

In [218]:
localizacao = gpd.GeoDataFrame(pd.concat([loc_bvsa, loc_dique6c]))
localizacao = localizacao.rename(columns={"Sensor":"sensor", "Latitude":"latitude", "Longitude":"longitude", "Altura":"altura"})

In [219]:
localizacao['sensor'] = localizacao['sensor'].apply(lambda x: x.replace("‐", '-').strip())

In [220]:
localizacao['id_sensor'] = localizacao['sensor'].map(dict_sensores)

In [221]:
localizacao = localizacao.reset_index(drop=True)
localizacao.index = localizacao.index + 1
localizacao

,sensor,N,E,latitude,longitude,altura,id_sensor
1,PZ-BM-BV-11,9.652626e+09,412507056.0,-3.142831,-51.787880,74.48,231
2,PZ-BM-BV-12,9.652626e+09,412507056.0,-3.142831,-51.787880,59.15,232
3,PZ-BM-BV-13,9.652636e+09,412475409.0,-3.142733,-51.788165,69.96,233
4,PZ-BM-BV-14,9.652636e+09,412475409.0,-3.142733,-51.788165,54.44,234
5,PZ-BM-BV-15,9.652622e+09,412336622.0,-3.142866,-51.789414,65.00,235
...,...,...,...,...,...,...,...
117,MV-D06C-04,9.652152e+09,413817837.0,-3.147122,-51.776087,65.00,138
118,RM-D06C-01,9.651928e+09,413655231.0,-3.149147,-51.777552,100.00,285
119,RM-D06C-02,9.651928e+09,413655231.0,-3.149147,-51.777552,100.00,286
120,RM-D06C-03,9.651928e+09,413655231.0,-3.149147,-51.777552,100.00,287


In [222]:
x_latitude = localizacao['latitude']
y_longitude = localizacao['longitude']
z_altura = localizacao['altura']

In [223]:
localizacao['geom'] = gpd.GeoSeries.from_xy(x =y_longitude, y=x_latitude, z=z_altura, crs='EPSG:4326')

In [224]:
localizacao = localizacao[['id_sensor', 'sensor', 'latitude', 'longitude', 'altura', 'geom']]

In [225]:
localizacao.head()

,id_sensor,sensor,latitude,longitude,altura,geom
1,231,PZ-BM-BV-11,-3.142831,-51.787880,74.48,POINT Z (-51.78788 -3.14283 74.48000)
2,232,PZ-BM-BV-12,-3.142831,-51.787880,59.15,POINT Z (-51.78788 -3.14283 59.15000)
3,233,PZ-BM-BV-13,-3.142733,-51.788165,69.96,POINT Z (-51.78816 -3.14273 69.96000)
4,234,PZ-BM-BV-14,-3.142733,-51.788165,54.44,POINT Z (-51.78816 -3.14273 54.44000)
5,235,PZ-BM-BV-15,-3.142866,-51.789414,65.00,POINT Z (-51.78941 -3.14287 65.00000)


In [226]:
SGBD_HOMOLOGACAO =  os.environ.get("DATABASE_URL")
HOST_HOMOLOGACAO =  os.environ.get("PGHOST")
USER_HOMOLOGACAO = os.environ.get("PGUSER")
DATABASE_HOMOLOGACAO =  os.environ.get("PGDATABASE")
PORT_HOMOLOGACAO =  os.environ.get("PGPORT")
PASSWORD_HOMOLOGACAO = os.environ.get("PGPASSWORD")
SQLALCHEMY_HOMOLOGACAO = os.environ.get("DATABASE_URL")

In [227]:
print("Criando conexão com o banco.")
try: 
    engine = create_engine(SQLALCHEMY_HOMOLOGACAO)
    conn = engine.connect()
    print('Conexão com banco de dados criada...')
except:
    print('Falha ao criar conexão com banco de dados.')

Criando conexão com o banco.


Conexão com banco de dados criada...


In [228]:
localizacao = localizacao.set_geometry('geom')

In [229]:
localizacao_ = localizacao[['id_sensor', 'sensor', 'latitude', 'longitude', 'altura']]

In [234]:
leituras = leituras.rename(columns={'index': 'id'})
localizacao = localizacao.rename(columns={'index': 'id'})

In [ ]:
#sensores.to_sql('sensores', schema='pesquisa', con=conn, if_exists='replace', index=True)
#unidade.to_sql('unidade', schema='pesquisa', con=conn, if_exists='replace', index=False)
#direcao_saida.to_sql('direcao_saida', schema='pesquisa', con=conn, if_exists='replace', index=False)
#area.to_sql('area', schema='pesquisa', con=conn, if_exists='replace', index=False)
#estado.to_sql('estado', schema='pesquisa', con=conn, if_exists='replace', index=False)
#leituras.to_sql('leituras', schema='pesquisa', con=conn, if_exists='replace', index=True)
#localizacao.to_postgis('localizacao', schema='pesquisa', con=conn, if_exists='replace', index=True)

In [ ]:
sensores.to_excel('../tabelas/sensores.xlsx')
unidades.to_excel('../tabelas/unidades.xlsx')
direcao_saida.to_excel('../tabelas/direcao_saida.xlsx')
leituras.to_excel('../tabelas/leituras.xlsx')
localizacao.to_excel('../tabelas/localizacao.xlsx')